<a href="https://colab.research.google.com/github/KyunghoonJeon/DeepLearningFromScratch/blob/main/firstNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
text="You say goodbye and I say hello."

In [ ]:
text=text.lower()
text=text.replace('.',' .')
text

In [ ]:
words=text.split(' ')
words

In [ ]:
word_to_id={}
id_to_word={}

In [ ]:
for word in words:
    if word not in word_to_id:
        new_id=len(word_to_id)
        word_to_id[word]=new_id
        id_to_word[new_id]=word

In [ ]:
id_to_word

In [ ]:
word_to_id

In [ ]:
id_to_word[1]

In [ ]:
word_to_id['hello']

In [ ]:
import numpy as np

In [ ]:
corpus=[word_to_id[w] for w in words]
corpus=np.array(corpus)
corpus

In [ ]:
def preprocess(text):
    text=text.lower()
    text=text.replace('.',' .')
    words=text.split(' ')
    word_to_id={}
    id_to_word={}
    for word in words:
        if word not in word_to_id:
            new_id=len(word_to_id)
            word_to_id[word]=new_id
            id_to_word[new_id]=word
    corpus=np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

In [ ]:
import sys
sys.path.append('..')

In [ ]:
text='You say goodbye and I say hello.'

In [ ]:
corpus, word_to_id, id_to_word=preprocess(text)

In [ ]:
print(corpus)

In [ ]:
print(id_to_word)

In [ ]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size=len(corpus)
    co_matrix=np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1,window_size + 1):
            left_idx=idx-i
            right_idx=idx+i

            if left_idx>=0:
                left_word_id=corpus[left_idx]
                co_matrix[word_id, left_word_id]+=1
            
            if right_idx<corpus_size:
                right_word_id=corpus[right_idx]
                co_matrix[word_id, right_word_id]+=1
    
    return co_matrix

In [ ]:
def cos_similarity(x,y,eps=1e-8):
    nx=x/np.sqrt(np.sum(x**2))+eps
    ny=y/np.sqrt(np.sum(y**2))+eps
    return np.dot(nx,ny)

In [ ]:
text='You say goodbye and I say hello.'
corpus,word_to_id, id_to_word=preprocess(text)
vocab_size=len(word_to_id)
C=create_co_matrix(corpus, vocab_size)

In [ ]:
c0=C[word_to_id['you']]
c1=C[word_to_id['i']]
print(cos_similarity(c0,c1))

여기서는 유사성이 크다고 얘기할 수 있다.

유사 단어의 랭킹 표시

In [ ]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    #검색어를 꺼낸다
    if query not in word_to_id:
        print('%s 을(를) 찾을 수 없습니다.' % query)
        return 
    
    print('\n[query]'+query)
    query_id=word_to_id[query]
    query_vec=word_matrix[query_id]

    #코사인 유사도 계산
    vocab_size=len(id_to_word)
    similarity=np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i]=cos_similarity(word_matrix[i], query_vec)
    
    #코사인 유사도를 기준으로 내림차순으로 출력
    count=0
    for i in (-1*similarity).argsort():
        if id_to_word[i]==query:
            continue
        print(' %s: %s' %(id_to_word[i], similarity[i]))

        count+=1
        if count>=top:
            return

argsort ==> 배열 인덱스의 정렬을 바꾸는 매서드(배열의 원소를 오름차순으로 정렬한 뒤에 반환은 배열의 인덱스)

In [ ]:
x=np.array([100,-20,2])
x.argsort()

In [ ]:
(-x).argsort()

In [ ]:
text='You say goodbye and I say hello.'
corpus,word_to_id, id_to_word=preprocess(text)
vocab_size=len(word_to_id)
C=create_co_matrix(corpus, vocab_size)

In [ ]:
most_similar('you', word_to_id, id_to_word, C, top=5)

말뭉치가 너무 작아서 원하는 결과가 나오지 못한다. 

In [ ]:
def ppmi(C, verbose=False, eps=1e-8):
    M=np.zeros_like(C,dtype=np.float32)
    N=np.sum(C)
    S=np.sum(C,axis=0)
    total=C.shape[0]*C.shape[1]
    cnt=0

    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi=np.log2(C[i,j]*N/(S[j]*S[i])+eps)
            M[i,j]=max(0,pmi)

            if verbose:
                cnt+=1
                if cnt % (total//100+1)==0:
                    print('%.1f%% 완료' %(100*cnt/total))
    
    return M

In [ ]:
text='You say goodbye and I say hello.'
corpus,word_to_id, id_to_word=preprocess(text)
vocab_size=len(word_to_id)
C=create_co_matrix(corpus, vocab_size)
W=ppmi(C)

In [ ]:
np.set_printoptions(precision=3)
print('동시발생 행렬')
print(C)
print('-'*50)
print('PPMI')
print(W)

말뭉치의 어휘 수가 증가함에 따라 각 단어 벡터의 차원 수도 증가한다...

In [ ]:
text='You say goodbye and I say hello.'
corpus,word_to_id, id_to_word=preprocess(text)
vocab_size=len(word_to_id)
C=create_co_matrix(corpus, vocab_size)
W=ppmi(C)

In [ ]:
U,S,V=np.linalg.svd(W)

In [ ]:
print(C[0])

In [ ]:
print(W[0])

In [ ]:
print(U[0])

In [ ]:
print(U[0,:2])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for word, word_id in word_to_id.items():
    plt.annotate(word,(U[word_id,0], U[word_id,1]))

plt.scatter(U[:,0], U[:,1], alpha=0.5)
plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

In [ ]:
from MyDrive import ptb